In [1]:
import random
from data import ImageDetectionsField, TextField, RawField
from data import COCO, DataLoader
from data.dataset import AP_Dataset
import evaluation
from evaluation import PTBTokenizer, Cider
from models.transformer import Transformer, MemoryAugmentedEncoder, MeshedDecoder, ScaledDotProductAttentionMemory
import torch
from torch.optim import Adam
from torch.optim.lr_scheduler import LambdaLR
from torch.nn import NLLLoss
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader as TorchDataLoader
from torch.utils.data import Dataset as TorchDataset
import argparse, os, pickle
from tqdm import tqdm
import numpy as np
import itertools
import multiprocessing
from shutil import copyfile
import h5py

random.seed(1234)
torch.manual_seed(1234)
np.random.seed(1234)

ImportError: numpy.core.multiarray failed to import

In [2]:
def text_progress(minibatch, text_field):
#         print(f"text_progress {0}", minibatch)
    batch_tokens = [batch['cap'] for batch in minibatch]
#         print(f"text_progress {1}", batch_tokens)
    padded_tokens = text_field.pad(batch_tokens)
#         print(f"text_progress {2}", padded_tokens)
    token_ids = text_field.numericalize(padded_tokens)
#         print(f"text_progress {3}", token_ids)
    return {"roi_feat": torch.from_numpy(np.array([batch['roi_feat'] for batch in minibatch])),
            "cap": token_ids}

In [3]:
ap_train_dataset = h5py.File("../Dataset/artpedia/artpedia_train2.hdf5", "r")

train_myidx = np.load('../Dataset/artpedia/train_myidx.npy')
   
ap_train_dataset = h5py.File("../Dataset/artpedia/artpedia_train2.hdf5", "r")

train_item2imgCapIdx = np.load("../Dataset/artpedia/train_item2imgCapIdx.npy", allow_pickle=True).item()

text_field = TextField(init_token='<bos>', eos_token='<eos>', fix_length=30, lower=True, tokenize='spacy',
                           remove_punctuation=True, nopoints=False)
text_field.vocab = pickle.load(open('vocab.pkl', 'rb'))
print("loading file: Done !")

loading file: Done !


In [4]:
artpedia_train = AP_Dataset(ap_train_dataset, train_item2imgCapIdx, text_field, \
                            max_detections=50, lower=True, remove_punctuation=True, tokenize='spacy')

In [6]:
train_data_loader = TorchDataLoader(artpedia_train, batch_size=1,
                                  collate_fn=lambda x: text_progress(x, artpedia_train.text_field))

for b in train_data_loader:
    print(b)
    break

-----0-----
{'roi_feat': tensor([[[8.9316e-01, 0.0000e+00, 1.5636e-03,  ..., 0.0000e+00,
          5.9531e+00, 9.1240e-01],
         [1.9096e-03, 0.0000e+00, 2.8516e+00,  ..., 5.0467e-02,
          5.3136e+00, 2.0535e+00],
         [0.0000e+00, 0.0000e+00, 1.6620e+00,  ..., 2.9239e-02,
          5.3941e+00, 2.2364e+00],
         ...,
         [0.0000e+00, 4.9739e-01, 5.4135e-02,  ..., 0.0000e+00,
          0.0000e+00, 8.5115e-02],
         [1.9324e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          5.9282e+00, 1.3733e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 2.1885e-01]]]), 'cap': tensor([[   2,   28,   11, 1234,    0, 7212,   13,   26,  742,  215,  469, 1223,
            0, 3914,   10,  543,    7, 4632,  110,    7, 3655, 6276,  551,  106,
          239,    7,   71, 2967, 1825,    3]])}


In [5]:
train_data_loader_shuffle = TorchDataLoader(artpedia_train, batch_size=5, shuffle=True,
                                  collate_fn=lambda x: text_progress(x, artpedia_train.text_field))

for b in train_data_loader_shuffle:
    print(b)
    break

-----3901-----
-----38-----
-----1438-----
-----4933-----
-----1718-----
{'roi_feat': tensor([[[0.0000e+00, 4.3145e-02, 0.0000e+00,  ..., 2.9388e+00,
          2.8484e-02, 1.3034e+00],
         [9.3100e-03, 1.7313e-01, 0.0000e+00,  ..., 6.9744e-01,
          2.5579e-01, 1.4116e-01],
         [0.0000e+00, 5.3775e-01, 2.3395e-01,  ..., 6.3382e-02,
          4.9877e+00, 2.4143e-02],
         ...,
         [1.4317e-01, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          1.9147e-02, 0.0000e+00],
         [6.7224e-02, 1.8240e-01, 0.0000e+00,  ..., 5.2549e+00,
          1.6558e-01, 1.1003e+00],
         [2.3813e-02, 6.0978e-01, 1.7086e-02,  ..., 1.6776e-01,
          0.0000e+00, 1.5174e-01]],

        [[3.5952e-03, 4.6582e-03, 0.0000e+00,  ..., 3.8823e-01,
          0.0000e+00, 1.3501e-01],
         [0.0000e+00, 0.0000e+00, 7.4356e-01,  ..., 1.4323e+00,
          8.1427e-03, 0.0000e+00],
         [0.0000e+00, 1.3064e+00, 1.4540e-02,  ..., 1.1177e+01,
          0.0000e+00, 9.5690e-03],
       

In [9]:
artpedia_train.__getitem__(849)

-----849-----


{'roi_feat': array([[2.0216746 , 0.        , 0.        , ..., 0.        , 1.3987926 ,
         0.        ],
        [0.30205244, 0.00586011, 0.02239603, ..., 0.        , 0.23666917,
         0.9295689 ],
        [1.1791258 , 0.5145521 , 2.6314425 , ..., 0.6239442 , 0.11476832,
         1.7293396 ],
        ...,
        [0.8122813 , 3.6445942 , 0.        , ..., 0.        , 0.32125932,
         0.05246392],
        [0.        , 2.8413603 , 3.27753   , ..., 0.        , 0.01809708,
         0.15031812],
        [0.06550153, 0.        , 0.03633639, ..., 0.        , 0.85828817,
         1.8566139 ]], dtype=float32),
 'cap': ['at',
  'the',
  'top',
  'is',
  'a',
  'rural',
  'background',
  'with',
  'a',
  'fortress',
  'and',
  'a',
  'walled',
  'city',
  'full',
  'of',
  'guard',
  'towers',
  'and',
  'bell',
  'towers',
  'along',
  'with',
  'other',
  'buildings',
  'based',
  'on',
  'famous',
  'buildings',
  'in',
  'romagna',
  'and',
  'venetia',
  'which',
  'bellini',
  'had